In [1]:
import pandas as pd
import numpy as np
import h5py
import os
import glob
import shutil

In [2]:
#convienence functions for .visit() and constants/globals/fixed params

#constants
sep='\n\n-------------------'
nameList=['fluxCo2','fluxMome','fluxTemp','fluxH2o','foot']#this may be added to due to the immense amount of data in
                                                           #each hdf5 file, watch for uniqueness of keys
avoidprintstats=['dp0p','dp01','dp02','dp03']

#fixed params
np.set_printoptions(threshold=500)#2000)

#globals, must watch carefully
fi=None
fd=None
cumulative_null=0
tot_sum=0
 
#convienence functions
def printname(name):#prints names of all groups/subgrups/dataset file names
    print(name)

def avoidnames(name):
    global avoidprintstats
    for avoidn in avoidprintstats:
        if avoidn in name:
            return True
    else: 
        return False;
    
    
def attrsearch(name,node):#prints the attributes of all groups/subgrups/dataset file names
    global showall
    attr = list(node.attrs.items())
    if avoidnames(name) and not showall:
        return
    print(name, node, attr)
    print(node.attrs)
    print('\n')
    
def appendto(name,node):#appends to our current generated file
    global fi,fd,searchval#must watch carefully
    if searchval in name and not searchval in node.parent.name:
        fd.copy(node, fd, name=name)
        
def printdat(name,node):#for data inspection of entire file
    if isinstance(node, h5py.Dataset) and not name=='readMe' and not name=='objDesc': 
        global showall
        if avoidnames(name) and not showall:
            return
        print(name, node, list(node.attrs.items()))
        print(node.value)
        nullnumcnt(node)
        print('\n')
        
def nullnumcnt(node):#converts the resulting np arr of values to dataframe, counts nulls and updates the running sums
    global cumulative_null,tot_sum
    if 'data' in node.name:
        if(avoidnames(node.name)):
            return
        df2=pd.DataFrame(node.value)
        nullnum=df2.isnull().values.ravel().sum()
        cumulative_null+=nullnum
        tot=len(df2.index)
        tot_sum+=tot
        print('Nullnum=%s tot=%s percent_null=%s'%(str(nullnum),str(tot),str((float(nullnum)/tot)*100)))
        
def read_process(sourcefilename):#read and process the individual hdf5 files, generated a separated file
    global fi,fd,mainfoldersel,searchval#must watch carefully
    fi=h5py.File(sourcefilename, 'r')
    fd=h5py.File(sourcefilename[:-3]+'_'+searchval+sourcefilename[-3:], 'w')#fd is file destination
    #print(sourcefilename[:-3]+'_'+nameList[0]+sourcefilename[-3:])
    fi.visititems(appendto)#need to modify this
    fd.copy(fi['readMe'], fd, name='readMe')#copy the readMe and objDesc from original file
    fd.copy(fi['objDesc'], fd, name='objDesc')
    fd.copy(fi[mainfoldersel+'/dp0p'], fd, name=mainfoldersel+'/dp0p')
    fd.copy(fi[mainfoldersel+'/dp01'], fd, name=mainfoldersel+'/dp01')
    fd.copy(fi[mainfoldersel+'/dp02'], fd, name=mainfoldersel+'/dp02')
    fd.copy(fi[mainfoldersel+'/dp03'], fd, name=mainfoldersel+'/dp03')
    fd.visit(printname)
    fd.visititems(printdat)
    fd.close()
    fi.close()
    
def reorganizeandzip():
    global searchval,basedir,foldername,searchterm
    if not os.path.exists(basedir+"/movefolder"):
        os.makedirs(basedir+"/movefolder")
        
    for fname in glob.glob(searchterm):
        for subfname in glob.glob(fname[:-4]+"/*.h5"):#search for h5 file in subdirectory we unzipped
            if not searchval in subfname:
                print("moving: "+subfname+" to "+basedir+"/movefolder")
                shutil.move(subfname, basedir+"/movefolder")
    for fname in glob.glob(searchterm+"*.zip"):
        #for subfname in glob.glob(fname[:-4]+"/*.zip"):
        print("zip moving: "+fname+" to "+basedir+"/movefolder")
        shutil.move(fname, basedir+"/movefolder")
        
    for fname in glob.glob(searchterm):
            shutil.make_archive(fname,'zip',fname)
    for fname in glob.glob(searchterm):
        if not '.zip' in fname:
            shutil.move(fname, basedir+"/movefolder")
        
    os.chdir('..')
    shutil.make_archive(foldername, 'zip', foldername)

In [3]:
#user variables
basedir="/Users/nickvancise"#base directory for reorganizing files
searchterm='NEON.D18.BARR'+'*'#hdf5 data file should contain this
foldername='NEON_eddy-flux-2'#folder name with all eddy files in it
showall=False
searchval=nameList[0]
mainfoldersel=searchterm[9:-1]
print(mainfoldersel)

BARR


In [4]:
os.chdir(foldername)

In [5]:
#splits hdf5 and prints structure and logging info
count=0#result should be 12, one for each month
for fname in glob.glob(searchterm):
    print(fname)
    shutil.unpack_archive(fname,fname[:-4],'zip')
    print('unzipped&extension removed:')
    print(fname[:-4])
    print('contains')
    for subfname in glob.glob(fname[:-4]+"/*.h5"):#search for h5 file in subdirectory we unzipped
        print(subfname)
        read_process(subfname)
    #break
    print('\n')
    print(sep)
    count+=1
print(count)
print('Nullnum_tot=%s tot_sum=%s tot_percent_null=%s'%(str(cumulative_null),str(tot_sum),str((float(cumulative_null)/tot_sum)*100)))

NEON.D18.BARR.DP4.00200.001.2018-08.basic.20190211T221124Z.zip
unzipped&extension removed:
NEON.D18.BARR.DP4.00200.001.2018-08.basic.20190211T221124Z
contains
NEON.D18.BARR.DP4.00200.001.2018-08.basic.20190211T221124Z/NEON.D18.BARR.DP4.00200.001.nsae.2018-08.basic.h5
BARR
BARR/dp01
BARR/dp01/data
BARR/dp01/data/soni
BARR/dp01/data/soni/000_040_01m
BARR/dp01/data/soni/000_040_01m/angZaxsErth
BARR/dp01/data/soni/000_040_01m/tempAir
BARR/dp01/data/soni/000_040_01m/tempSoni
BARR/dp01/data/soni/000_040_01m/veloXaxsYaxsErth
BARR/dp01/data/soni/000_040_01m/veloXaxsErth
BARR/dp01/data/soni/000_040_01m/veloYaxsErth
BARR/dp01/data/soni/000_040_01m/veloZaxsErth
BARR/dp01/data/soni/000_040_30m
BARR/dp01/data/soni/000_040_30m/angZaxsErth
BARR/dp01/data/soni/000_040_30m/tempAir
BARR/dp01/data/soni/000_040_30m/tempSoni
BARR/dp01/data/soni/000_040_30m/veloXaxsYaxsErth
BARR/dp01/data/soni/000_040_30m/veloXaxsErth
BARR/dp01/data/soni/000_040_30m/veloYaxsErth
BARR/dp01/data/soni/000_040_30m/veloZaxsErth


BARR/dp01/qfqm/presBaro/000_035_30m
BARR/dp01/qfqm/radiNet
BARR/dp01/qfqm/radiNet/000_040_01m
BARR/dp01/qfqm/radiNet/000_040_30m
BARR/dp01/qfqm/tempAirLvl
BARR/dp01/qfqm/tempAirLvl/000_010_01m
BARR/dp01/qfqm/tempAirLvl/000_010_01m/temp
BARR/dp01/qfqm/tempAirLvl/000_010_30m
BARR/dp01/qfqm/tempAirLvl/000_010_30m/temp
BARR/dp01/qfqm/tempAirLvl/000_020_01m
BARR/dp01/qfqm/tempAirLvl/000_020_01m/temp
BARR/dp01/qfqm/tempAirLvl/000_020_30m
BARR/dp01/qfqm/tempAirLvl/000_020_30m/temp
BARR/dp01/qfqm/tempAirLvl/000_030_01m
BARR/dp01/qfqm/tempAirLvl/000_030_01m/temp
BARR/dp01/qfqm/tempAirLvl/000_030_30m
BARR/dp01/qfqm/tempAirLvl/000_030_30m/temp
BARR/dp01/qfqm/isoH2o
BARR/dp01/qfqm/isoH2o/000_030_09m
BARR/dp01/qfqm/isoH2o/000_030_09m/rtioMoleWetH2o
BARR/dp01/qfqm/isoH2o/000_030_09m/dlta18OH2o
BARR/dp01/qfqm/isoH2o/000_030_09m/temp
BARR/dp01/qfqm/isoH2o/000_030_09m/rhEnvHut
BARR/dp01/qfqm/isoH2o/000_030_09m/tempEnvHut
BARR/dp01/qfqm/isoH2o/000_030_09m/rtioMoleDryH2o
BARR/dp01/qfqm/isoH2o/000_030_09m

BARR/dp01/ucrt/tempAirLvl/000_030_30m/temp
BARR/dp01/ucrt/isoH2o
BARR/dp01/ucrt/isoH2o/000_030_09m
BARR/dp01/ucrt/isoH2o/000_030_09m/rtioMoleWetH2o
BARR/dp01/ucrt/isoH2o/000_030_09m/dlta18OH2o
BARR/dp01/ucrt/isoH2o/000_030_09m/temp
BARR/dp01/ucrt/isoH2o/000_030_09m/rhEnvHut
BARR/dp01/ucrt/isoH2o/000_030_09m/tempEnvHut
BARR/dp01/ucrt/isoH2o/000_030_09m/rtioMoleDryH2o
BARR/dp01/ucrt/isoH2o/000_030_09m/presEnvHut
BARR/dp01/ucrt/isoH2o/000_030_09m/rtioMoleWetH2oEnvHut
BARR/dp01/ucrt/isoH2o/000_030_09m/dlta2HH2o
BARR/dp01/ucrt/isoH2o/000_030_09m/pres
BARR/dp01/ucrt/isoH2o/000_030_30m
BARR/dp01/ucrt/isoH2o/000_030_30m/rtioMoleWetH2o
BARR/dp01/ucrt/isoH2o/000_030_30m/dlta18OH2o
BARR/dp01/ucrt/isoH2o/000_030_30m/temp
BARR/dp01/ucrt/isoH2o/000_030_30m/rhEnvHut
BARR/dp01/ucrt/isoH2o/000_030_30m/tempEnvHut
BARR/dp01/ucrt/isoH2o/000_030_30m/rtioMoleDryH2o
BARR/dp01/ucrt/isoH2o/000_030_30m/presEnvHut
BARR/dp01/ucrt/isoH2o/000_030_30m/rtioMoleWetH2oEnvHut
BARR/dp01/ucrt/isoH2o/000_030_30m/dlta2HH2o


BARR/dp04/data/fluxCo2/nsae <HDF5 dataset "nsae": shape (1248,), type "|V58"> [('unit', array([b'umolCo2 m-2 s-1'], dtype='|S16'))]
[(b'2018-08-03T00:00:00.000Z', b'2018-08-03T00:29:59.000Z', nan)
 (b'2018-08-03T00:30:00.000Z', b'2018-08-03T00:59:59.000Z', nan)
 (b'2018-08-03T01:00:00.000Z', b'2018-08-03T01:29:59.000Z', nan) ...
 (b'2018-08-31T22:30:00.000Z', b'2018-08-31T22:59:59.000Z', nan)
 (b'2018-08-31T23:00:00.000Z', b'2018-08-31T23:29:59.000Z', nan)
 (b'2018-08-31T23:30:00.000Z', b'2018-08-31T23:59:59.000Z', nan)]
Nullnum=1218 tot=1248 percent_null=97.59615384615384


BARR/dp04/data/fluxCo2/stor <HDF5 dataset "stor": shape (1248,), type "|V58"> [('unit', array([b'umolCo2 m-2 s-1'], dtype='|S16'))]
[(b'2018-08-03T00:00:00.000Z', b'2018-08-03T00:29:59.000Z',         nan)
 (b'2018-08-03T00:30:00.000Z', b'2018-08-03T00:59:59.000Z',         nan)
 (b'2018-08-03T01:00:00.000Z', b'2018-08-03T01:29:59.000Z',         nan)
 ...
 (b'2018-08-31T22:30:00.000Z', b'2018-08-31T22:59:59.000Z', -0

BARR/dp01/qfqm/h2oStor/co2High_02m/presEnvHut
BARR/dp01/qfqm/h2oStor/co2High_02m/rtioMoleWetH2oEnvHut
BARR/dp01/qfqm/h2oStor/co2High_02m/pres
BARR/dp01/qfqm/h2oStor/co2Zero_30m
BARR/dp01/qfqm/h2oStor/co2Zero_30m/rtioMoleWetH2o
BARR/dp01/qfqm/h2oStor/co2Zero_30m/frt00
BARR/dp01/qfqm/h2oStor/co2Zero_30m/temp
BARR/dp01/qfqm/h2oStor/co2Zero_30m/rhEnvHut
BARR/dp01/qfqm/h2oStor/co2Zero_30m/tempEnvHut
BARR/dp01/qfqm/h2oStor/co2Zero_30m/rtioMoleDryH2o
BARR/dp01/qfqm/h2oStor/co2Zero_30m/presEnvHut
BARR/dp01/qfqm/h2oStor/co2Zero_30m/rtioMoleWetH2oEnvHut
BARR/dp01/qfqm/h2oStor/co2Zero_30m/pres
BARR/dp01/qfqm/h2oStor/co2Med_02m
BARR/dp01/qfqm/h2oStor/co2Med_02m/rtioMoleWetH2o
BARR/dp01/qfqm/h2oStor/co2Med_02m/frt00
BARR/dp01/qfqm/h2oStor/co2Med_02m/temp
BARR/dp01/qfqm/h2oStor/co2Med_02m/rhEnvHut
BARR/dp01/qfqm/h2oStor/co2Med_02m/tempEnvHut
BARR/dp01/qfqm/h2oStor/co2Med_02m/rtioMoleDryH2o
BARR/dp01/qfqm/h2oStor/co2Med_02m/presEnvHut
BARR/dp01/qfqm/h2oStor/co2Med_02m/rtioMoleWetH2oEnvHut
BARR/dp01/q

...

BARR/dp0p/data/rhSens/003_000
BARR/dp0p/data/irgaTurb
BARR/dp0p/data/irgaTurb/000_040
BARR/dp0p/data/crdCo2ValvLvl
BARR/dp0p/data/crdCo2ValvLvl/702_000
BARR/dp0p/data/valvLeakHeatTurb
BARR/dp0p/data/valvLeakHeatTurb/000_040
BARR/dp0p/data/gasRefe
BARR/dp0p/data/gasRefe/704_000
BARR/dp0p/data/gasRefe/705_000
BARR/dp0p/data/gasRefe/706_000
BARR/dp0p/data/gasRefe/710_000
BARR/dp0p/data/gasRefe/712_000
BARR/dp0p/data/gasRefe/702_000
BARR/dp0p/data/gasRefe/713_000
BARR/dp0p/data/gasRefe/714_000
BARR/dp0p/data/presValiRegInStor
BARR/dp0p/data/presValiRegInStor/710_000
BARR/dp0p/data/presValiRegInStor/711_000
BARR/dp0p/data/presValiRegInStor/712_000
BARR/dp0p/data/presValiRegInStor/713_000
BARR/dp0p/data/presValiRegInStor/714_000
BARR/dp0p/data/presValiRegInStor/709_000
BARR/dp0p/data/valvValiNemaTurb
BARR/dp0p/data/valvValiNemaTurb/000_040
BARR/dp0p/data/mfcSampTurb
BARR/dp0p/data/mfcSampTurb/000_040
BARR/dp0p/data/valvValiStor
BARR/dp0p/data/valvValiStor/703_000
BARR/dp0p/data/valvAux
BARR/

BARR/dp01/qfqm/co2Stor/000_040_30m/tempEnvHut
BARR/dp01/qfqm/co2Stor/000_040_30m/presEnvHut
BARR/dp01/qfqm/co2Stor/000_040_30m/rtioMoleWetH2oEnvHut
BARR/dp01/qfqm/co2Stor/000_040_30m/pres
BARR/dp01/qfqm/co2Stor/000_040_02m
BARR/dp01/qfqm/co2Stor/000_040_02m/frt00
BARR/dp01/qfqm/co2Stor/000_040_02m/rtioMoleDryCo2
BARR/dp01/qfqm/co2Stor/000_040_02m/temp
BARR/dp01/qfqm/co2Stor/000_040_02m/rhEnvHut
BARR/dp01/qfqm/co2Stor/000_040_02m/rtioMoleWetCo2
BARR/dp01/qfqm/co2Stor/000_040_02m/tempEnvHut
BARR/dp01/qfqm/co2Stor/000_040_02m/presEnvHut
BARR/dp01/qfqm/co2Stor/000_040_02m/rtioMoleWetH2oEnvHut
BARR/dp01/qfqm/co2Stor/000_040_02m/pres
BARR/dp01/qfqm/co2Stor/co2High_02m
BARR/dp01/qfqm/co2Stor/co2High_02m/frt00
BARR/dp01/qfqm/co2Stor/co2High_02m/rtioMoleDryCo2
BARR/dp01/qfqm/co2Stor/co2High_02m/temp
BARR/dp01/qfqm/co2Stor/co2High_02m/rhEnvHut
BARR/dp01/qfqm/co2Stor/co2High_02m/rtioMoleWetCo2
BARR/dp01/qfqm/co2Stor/co2High_02m/tempEnvHut
BARR/dp01/qfqm/co2Stor/co2High_02m/presEnvHut
BARR/dp01/qf

BARR/dp0p/data/mfcSampStor/700_000
BARR/dp0p/data/crdH2oValvVali
BARR/dp0p/data/crdH2oValvVali/700_000
BARR/dp0p/data/presValiRegOutStor
BARR/dp0p/data/presValiRegOutStor/710_000
BARR/dp0p/data/presValiRegOutStor/711_000
BARR/dp0p/data/presValiRegOutStor/712_000
BARR/dp0p/data/presValiRegOutStor/713_000
BARR/dp0p/data/presValiRegOutStor/714_000
BARR/dp0p/data/presValiRegOutStor/709_000
BARR/dp0p/data/pumpStor
BARR/dp0p/data/pumpStor/700_010
BARR/dp0p/data/pumpStor/700_020
BARR/dp0p/data/pumpStor/700_030
BARR/dp0p/data/pumpStor/700_040
BARR/dp0p/data/pumpStor/700_000
BARR/dp0p/data/presInlt
BARR/dp0p/data/presInlt/000_010
BARR/dp0p/data/presInlt/000_020
BARR/dp0p/data/presInlt/000_030
BARR/dp0p/data/presInlt/000_040
BARR/dp0p/data/crdH2o
BARR/dp0p/data/crdH2o/000_010
BARR/dp0p/data/crdH2o/000_020
BARR/dp0p/data/crdH2o/000_030
BARR/dp0p/data/crdH2o/000_040
BARR/dp0p/data/crdH2o/h2oHigh
BARR/dp0p/data/crdH2o/h2oMed
BARR/dp0p/data/crdH2o/h2oLow
BARR/dp0p/qfqm
BARR/dp0p/qfqm/soni
BARR/dp0p/

BARR/dp01/qfqm/co2Stor/co2Med_02m/tempEnvHut
BARR/dp01/qfqm/co2Stor/co2Med_02m/presEnvHut
BARR/dp01/qfqm/co2Stor/co2Med_02m/rtioMoleWetH2oEnvHut
BARR/dp01/qfqm/co2Stor/co2Med_02m/pres
BARR/dp01/qfqm/co2Stor/co2Low_30m
BARR/dp01/qfqm/co2Stor/co2Low_30m/frt00
BARR/dp01/qfqm/co2Stor/co2Low_30m/rtioMoleDryCo2
BARR/dp01/qfqm/co2Stor/co2Low_30m/temp
BARR/dp01/qfqm/co2Stor/co2Low_30m/rhEnvHut
BARR/dp01/qfqm/co2Stor/co2Low_30m/rtioMoleWetCo2
BARR/dp01/qfqm/co2Stor/co2Low_30m/tempEnvHut
BARR/dp01/qfqm/co2Stor/co2Low_30m/presEnvHut
BARR/dp01/qfqm/co2Stor/co2Low_30m/rtioMoleWetH2oEnvHut
BARR/dp01/qfqm/co2Stor/co2Low_30m/pres
BARR/dp01/qfqm/co2Stor/000_010_02m
BARR/dp01/qfqm/co2Stor/000_010_02m/frt00
BARR/dp01/qfqm/co2Stor/000_010_02m/rtioMoleDryCo2
BARR/dp01/qfqm/co2Stor/000_010_02m/temp
BARR/dp01/qfqm/co2Stor/000_010_02m/rhEnvHut
BARR/dp01/qfqm/co2Stor/000_010_02m/rtioMoleWetCo2
BARR/dp01/qfqm/co2Stor/000_010_02m/tempEnvHut
BARR/dp01/qfqm/co2Stor/000_010_02m/presEnvHut
BARR/dp01/qfqm/co2Stor/000

BARR/dp0p/qfqm/irgaStor/000_010
BARR/dp0p/qfqm/irgaStor/co2High
BARR/dp0p/qfqm/irgaStor/co2Med
BARR/dp0p/qfqm/irgaStor/000_030
BARR/dp0p/qfqm/irgaStor/000_020
BARR/dp0p/qfqm/irgaStor/000_040
BARR/dp0p/qfqm/irgaStor/co2Zero
BARR/dp0p/qfqm/irgaStor/co2Low
BARR/dp0p/qfqm/crdCo2
BARR/dp0p/qfqm/crdCo2/000_010
BARR/dp0p/qfqm/crdCo2/000_020
BARR/dp0p/qfqm/crdCo2/000_030
BARR/dp0p/qfqm/crdCo2/000_040
BARR/dp0p/qfqm/crdCo2/co2Arch
BARR/dp0p/qfqm/crdCo2/co2High
BARR/dp0p/qfqm/crdCo2/co2Med
BARR/dp0p/qfqm/crdCo2/co2Low
BARR/dp0p/qfqm/envHut
BARR/dp0p/qfqm/envHut/700_000
BARR/dp0p/qfqm/amrs
BARR/dp0p/qfqm/amrs/000_040
BARR/dp0p/qfqm/mfcValiTurb
BARR/dp0p/qfqm/mfcValiTurb/000_040
BARR/dp0p/qfqm/mfcValiStor
BARR/dp0p/qfqm/mfcValiStor/700_000
BARR/dp0p/qfqm/rhSens
BARR/dp0p/qfqm/rhSens/000_040
BARR/dp0p/qfqm/rhSens/003_000
BARR/dp0p/qfqm/irgaTurb
BARR/dp0p/qfqm/irgaTurb/000_040
BARR/dp0p/qfqm/gasRefe
BARR/dp0p/qfqm/gasRefe/704_000
BARR/dp0p/qfqm/gasRefe/705_000
BARR/dp0p/qfqm/gasRefe/706_000
BARR/dp0

In [6]:
reorganizeandzip()

moving: NEON.D18.BARR.DP4.00200.001.2018-08.basic.20190211T221124Z/NEON.D18.BARR.DP4.00200.001.nsae.2018-08.basic.h5 to /Users/nickvancise/movefolder
moving: NEON.D18.BARR.DP4.00200.001.2018-12.basic.20190315T164101Z/NEON.D18.BARR.DP4.00200.001.nsae.2018-12.basic.h5 to /Users/nickvancise/movefolder
moving: NEON.D18.BARR.DP4.00200.001.2018-03.basic.20190201T164013Z/NEON.D18.BARR.DP4.00200.001.nsae.2018-03.basic.h5 to /Users/nickvancise/movefolder
moving: NEON.D18.BARR.DP4.00200.001.2018-02.basic.20190201T165058Z/NEON.D18.BARR.DP4.00200.001.nsae.2018-02.basic.h5 to /Users/nickvancise/movefolder
moving: NEON.D18.BARR.DP4.00200.001.2018-10.basic.20190221T153024Z/NEON.D18.BARR.DP4.00200.001.nsae.2018-10.basic.h5 to /Users/nickvancise/movefolder
moving: NEON.D18.BARR.DP4.00200.001.2018-05.basic.20190201T163732Z/NEON.D18.BARR.DP4.00200.001.nsae.2018-05.basic.h5 to /Users/nickvancise/movefolder
moving: NEON.D18.BARR.DP4.00200.001.2018-11.basic.20190307T172445Z/NEON.D18.BARR.DP4.00200.001.nsae.

In [7]:
for fname in glob.glob(searchterm):
    print(fname)

NEON.D18.BARR.DP4.00200.001.nsae.2018-01.basic.h5
NEON.D18.BARR.DP4.00200.001.nsae.2018-01.basic_fluxCo2.h5_tester
